In [1]:
import math as m
import os
from pathlib import Path
import numpy as np
from numpy.polynomial.polynomial import Polynomial
import matplotlib.pyplot as plt
import vice

In [2]:
OUTPUT_DIR = Path('../src/data/onezone/threeinfall')
END_TIME = 13.2
DT = 0.01
SIMTIME = np.arange(0, END_TIME+DT, DT)

In [3]:
from vice.yields.presets import JW20
vice.yields.sneia.settings['fe'] *= 0.4
vice.yields.ccsne.settings['fe'] *= 0.3
vice.yields.ccsne.settings['o'] *= 0.3

In [ ]:
class exponential:
    def __init__(self, timescale=1, coeff=1):
        self.timescale = timescale
        self.coeff = coeff
        
    def __call__(self, time):
        return self.coeff / self.timescale * m.exp(-time / self.timescale)

class threeinfall:
    def __init__(self, radius, first_timescale=0.103, second_timescale=0.411, 
                 third_timescale=0.15, second_onset=4.085, third_onset=11,
                 thin_to_thick=3.472, thin_disk_ratio=2.33):
        self.second_onset = second_onset
        self.third_onset = third_onset
        self.first = exponential(
            timescale = first_timescale,
            coeff = 1 / (1 - m.exp(-END_TIME / first_timescale))
        )
        self.second = exponential(
            timescale = second_timescale,
            coeff = thin_to_thick / (1 - m.exp(
                -(self.third_onset - self.second_onset) / second_timescale
            ))
        )
        self.third = exponential(
            timescale = third_timescale,
            coeff = thin_to_thick / (1 - m.exp(
                -(END_TIME - self.third_onset) / third_timescale
            ))
        )
        
    def __call__(self, time):
        ifr = self.first(time)
        if time >= self.second_onset:
            ifr += self.second(time - self.second_onset)
        if time >= self.third_onset:
            ifr += self.third(time - self.third_onset)
        return ifr